# Download dos casos de Leishmaniose Visceral 

#### Os casos de Leixhmaniose Visceral foram baixados a partir do repositório do Sistema de Informação de Agravos de Notificação (SINAN),  que é alimentado, principalmente, pela notificação e investigação de casos de doenças e agravos que constam da lista nacional de doenças de notificação compulsória.  O downlod foi feito a partir do Package Python PySUS, criado para auxiliar no acesso e manipulação de base de dados publicadas pelo DATASUS.

#### Maiores informações em:

 - [SINAN] (https://portalsinan.saude.gov.br/sinan-net)
 - [Leishmaniose Visceral] (https://portalsinan.saude.gov.br/leishmaniose-visceral)
 - [PySUS] (https://pysus.readthedocs.io/en/latest/)


#### 1 - Importando o método SINAN do pacote PySUS

In [46]:
from pysus.ftp.databases.sinan import SINAN

#### 2 - Carregando os dados a partir do servidor do DataSUS

In [3]:
sinan = SINAN().load()

#### 3 - Lista os arquivos disponíveis para Leishmaniose Visceral

In [4]:
arquivos = sinan.get_files(dis_code=["LEIV"])

#### 4 - Download dos arquivos disponíveis

In [5]:
parquets = sinan.download(arquivos)

100%|██████████| 537k/537k [00:00<00:00, 387MB/s]


#### 5 - Converte os arquivos parquet em uma lista de dataframes

In [7]:
dataframes = [parquet.to_dataframe() for parquet in parquets]

#### 6 - Concatena em um único dataframe selecionando apenas as colunas de interesse

In [6]:
import pandas as pd

In [8]:
selected_columns = ['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'ID_MUNICIP']
selected_dfs = [df[selected_columns] for df in dataframes]
df = pd.concat(selected_dfs, ignore_index=True, axis=0)

#### 7 - Corrige os tipos dos campos de data e numéricos, e decodifica e o campo da semana da notificação

In [12]:
import numpy as np

In [10]:
# Cria a função convert_week para decodificar a coluna "SEM_NOT" 
def convert_week(x):
        try:
            w = int(x) % 100
        except ValueError:
            w = np.nan
        return w


In [13]:
    for cname in df.columns:
        df[cname].replace("", np.nan, inplace=True)
        if cname.startswith(("NU", "ID")):
            try:
                df[cname] = pd.to_numeric(df[cname])
            except ValueError as e:
                print(f"Column {cname} could not be converted to numeric: {e}")
                # certain IDs can be alphanumerical
                pass
        elif cname.startswith("DT"):
            try:
                df[cname] = pd.to_datetime(df[cname])
            except ValueError as e:
                print(f"Column {cname} could not be converted to date: {e}")
        elif cname.startswith("SEM"):
            df[cname] = df[cname].map(convert_week)

/tmp/ipykernel_3143/711638361.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[cname].replace("", np.nan, inplace=True)
/tmp/ipykernel_3143/711638361.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

#### 8 - Agrupa e contando os registros por data e município

In [14]:
counts = df.groupby(['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'ID_MUNICIP']).size().reset_index(name='count')

#### 9 - Ligação com a tabela de municípios importada com o pacote Geobr
#### Até o ano de 2006, o código do município apresenta 7 dígitos, depois de 2006 o código do município apresenta 6 dígitos. Desta maneira o merge será dividido em duas etapas

In [17]:
from geobr import read_municipality

In [18]:
mun = read_municipality(code_muni="all", year=2022)

In [27]:
# Converte a coluna 'code_muni' de float para int em uma nova coluna 'ID_MUNICIP'
mun['ID_MUNICIP_1'] = mun['code_muni'].astype('Int64')

# Remover o último dígito da coluna 'ID_MUNICIP'
mun['ID_MUNICIP_2'] = mun['ID_MUNICIP_1'].astype(str).str[:-1].astype(int)

In [28]:
selecao_1 = (counts['DT_NOTIFIC'] >= '2000-01-01') & (counts['DT_NOTIFIC'] <= '2006-12-31')
counts_1 = counts[selecao_1]

,DT_NOTIFIC,SEM_NOT,NU_ANO,ID_MUNICIP,count
0,2000-01-01,99.0,2000,2704302,1
1,2000-01-02,0.0,2000,2101608,1
2,2000-01-02,0.0,2000,2103000,2
3,2000-01-02,0.0,2000,2103406,1
4,2000-01-02,0.0,2000,2109452,1
...,...,...,...,...,...
28672,2006-12-31,7.0,2006,2304400,1
28673,2006-12-31,7.0,2006,2312908,1
28674,2006-12-31,7.0,2006,3106200,1
28675,2006-12-31,7.0,2006,3550308,2


In [29]:
selecao_2 = (counts['DT_NOTIFIC'] >= '2007-01-01') & (counts['DT_NOTIFIC'] <= '2022-12-31')
counts_2 = counts[selecao_2]

,DT_NOTIFIC,SEM_NOT,NU_ANO,ID_MUNICIP,count
28677,2007-01-01,1.0,2007,150470,1
28678,2007-01-01,1.0,2007,230230,1
28679,2007-01-01,1.0,2007,280030,1
28680,2007-01-02,1.0,2007,150130,1
28681,2007-01-02,1.0,2007,150210,1
...,...,...,...,...,...
134384,2022-12-31,52.0,2022,240810,1
134385,2022-12-31,52.0,2022,312090,1
134386,2022-12-31,52.0,2022,355500,1
134387,2022-12-31,52.0,2022,500270,1


In [33]:
# Junção dos dataframes 'counts' e 'mun' mantendo todos os registros mesmo quando não houver correspondência
merged_1 = counts_1.merge(mun, left_on='ID_MUNICIP', right_on='ID_MUNICIP_1', how='outer', indicator=True)
merged_2 = counts_2.merge(mun, left_on='ID_MUNICIP', right_on='ID_MUNICIP_2', how='outer', indicator=True)


In [34]:
merged = pd.concat([merged_1, merged_2])
merged

,DT_NOTIFIC,SEM_NOT,NU_ANO,ID_MUNICIP_x,count,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,geometry,ID_MUNICIP_y,ID_MUNICIP_1,ID_MUNICIP_2,_merge
0,NaT,NaN,<NA>,<NA>,NaN,1100015.0,Alta Floresta D'oeste,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-61.96836 -12.13407, -61.96827...",110001,1100015,110001,right_only
1,NaT,NaN,<NA>,<NA>,NaN,1100023.0,Ariquemes,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-63.18281 -10.13892, -63.18615...",110002,1100023,110002,right_only
2,NaT,NaN,<NA>,<NA>,NaN,1100031.0,Cabixi,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-60.70518 -13.32474, -60.70904...",110003,1100031,110003,right_only
3,NaT,NaN,<NA>,<NA>,NaN,1100049.0,Cacoal,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-61.34740 -11.50562, -61.34569...",110004,1100049,110004,right_only
4,NaT,NaN,<NA>,<NA>,NaN,1100056.0,Cerejeiras,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-60.82417 -13.11156, -60.82289...",110005,1100056,110005,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108776,2022-12-14,50.0,2022,530010,1.0,5300108.0,Brasília,53.0,DF,Distrito Federal,5.0,Centro Oeste,"MULTIPOLYGON (((-47.30850 -16.04997, -47.30997...",530010,5300108,530010,both
108777,2022-12-26,52.0,2022,530010,1.0,5300108.0,Brasília,53.0,DF,Distrito Federal,5.0,Centro Oeste,"MULTIPOLYGON (((-47.30850 -16.04997, -47.30997...",530010,5300108,530010,both
108778,2022-12-27,52.0,2022,530010,1.0,5300108.0,Brasília,53.0,DF,Distrito Federal,5.0,Centro Oeste,"MULTIPOLYGON (((-47.30850 -16.04997, -47.30997...",530010,5300108,530010,both
108779,2022-12-28,52.0,2022,530010,3.0,5300108.0,Brasília,53.0,DF,Distrito Federal,5.0,Centro Oeste,"MULTIPOLYGON (((-47.30850 -16.04997, -47.30997...",530010,5300108,530010,both


#### 10 - Exportando em CSV

In [38]:
# Reordenando as colunas
sinan_leishmaniose_2000_2022 = merged[['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'code_muni', 'name_muni', 'abbrev_state', 'name_state', 'name_region', 'count', 'geometry']]

# Renomeando as colunas
sinan_leishmaniose_2000_2022.columns = ['data', 'semana', 'ano','geocodigo', 'municipio',  'uf_sigla', 'uf', 'regiao', 'casos', 'geometry']


In [39]:
# Exportando para CSV
sinan_leishmaniose_2000_2022.to_csv('sinan_leishmaniose_2000_2022.csv',sep=';')

#### 11 - Exportando em GeoJason

In [43]:
# Importando geopandas
import geopandas as gpd

In [44]:
# Convertendo para GeoDataFrame
gdf = gpd.GeoDataFrame(sinan_leishmaniose_2000_2022)

In [45]:
# Exportanod para GeoJSON
gdf.to_file('sinan_leishmaniose_2000_2022.geojson', driver='GeoJSON')